# Entrega 3 - KNN y Naive Bayes

### Grupo 21:
     - Lucía Bouza  C.I 42897970



## 1. Objetivo

El objetivo de esta tarea se divide principalmente en dos. Por un lado es necesario estudiar y entender el conjunto de datos con el que se trabajará (Adult) y tomar decisiones para su manejo. Por otro lado se pide construir clasificadores basados en los algoritmos KNN y Naive Bayes, y comparar la performance con las implementaciones de Scikit-learn sobre el conjunto de datos. 

Se medirá la performance de los diferentes métodos utilizando las métricas de accurancy, precisión, recall y F1.

Para todas las implementaciones se utilizará validación cruzada para ajuste de los parámetros con un conjunto de entrenamiento, y luego sobre un conjunto de test, determinar la performance final de los algoritmos. 

## 2. Parte A


    
## 2.1 Caracterísiticas del Dataset

El conjunto de datos Adult consta de información extraída de una base de datos de censo de personas. En el dataset se detallan 14 características de entrada y una clasificación que indica si los ingresos anuales son mayores o menores a 50.000 dólares. 

El dataset cuenta con 48842 registros, de los cuales 45222 tienen todos los valores para todos los atributos. 

La distribución de la clase objetivo es:
 * clase >50K: 23.93% o 24,78% si se quitan los registros con valores faltantes.
 * clase <=50K: 76.07% o 75.22% si se quitan los registros con valores faltantes.

los atributos de entrada son los siguientes, pudiendo tomar los valores:

* Edad: entero. 
* Relación laboral: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
* Peso Final: continuous. 
* Educación: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. 
* Años de educación: entero. 
* Estado civil: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. 
* ocupación: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. 
* Relación en la familia: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. 
* Raza: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. 
* Sexo: Female, Male. 
* Capital ganado: entero. 
* Capital perdido: entero. 
* Horas por semana: entero. 
* País natal: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

El peso final es un valor que indica la cantidad de personas que el censo estima que representa ese registro. 

Ejecutemos la función de pandas Data profiling para obtener más datos acerca de los atributos, su distribución, valores, relaciones, etc. Tener en cuenta que en el reporte no detecta los valores faltantes porque en el dataset aparecen como "?" y el reporte lo considera un valor válido. 



In [1]:
import pandas as pd
import pandas_profiling
from sklearn import feature_selection, model_selection, preprocessing
import preprocesamiento
from NaiveBayes import NB
from KNN import KNN

#Carga de Datos
columnas =['Edad','Relación_laboral', 'Peso_Final', 'Educación', 'Años_de_educación', 'Estado_civil', 'Ocupación', 'Relación_en_la_familia', 'Raza', 'Sexo', 'Capital_ganado', 'Capital_perdido', 'Horas_por_semana', 'País_natal', 'Salida' ]
dataset = pd.read_csv('./Data/adult.data.csv', sep=',', names = columnas)

#generación de informe
pandas_profiling.ProfileReport(dataset).to_widgets() 



    
## 2.2 Análisis del dataset

Analizando los atributos, observamos que el atributo "Relación en la familia" bien puede deducirse de los atributos "Estado civil" y "Sexo". Es por eso que decidimos no tener en cuenta este atributo.
El atributo "Años de educación" es una representación numérica del campo "Educación". Se decide no tener en cuenta el atributo Educación, y aprovechar el "Años de educación" que ya es numérico y además interesa el orden en los valores.

Con respecto al atributo "Peso final", según su descripción, no parece ser un atributo relevante para el aprendizaje de la clasificación, sino más bien un peso relativo referido de la instancia en comparación a la población total. Al ver la correlación con la clase de Salida en el informe de Data profiling, vemos que el valor para Chi-squared es nulo. Decidimos no tener en cuenta este atributo.

Las columnas Capital Ganado y Capital perdido tienen un alto contenido de ceros (91,7% y 95,3% respectivamente). No sabemos si esto se debe a algún problema en la recolección de los datos, si las personas que contestaron el censo no tenían claro como contestar esta pregunta o si son valores reales. Ejecutaremos los algoritmos con estas columnas y sin ellas para evaluar si conviene utilizarlas o no. Igualmente vemos que, sobretodo el dato de Capital Ganado tiene una importante correlación con la clase de Salida.

Los valores faltantes se dan para los atributos Relación laboral (1836), ocupación (1843) y país natal (583). Sorprende que los atributos numéricos no tengan atributos faltantes, es por eso que volvemos a sospechar de las columnas con información referida al capital ganado y perdido. 

Observamos que el atributo Raza y País_Natal tienen muy baja correlación con la clase de Salida. 



    
## 2.3 Preprocesamiento de datos

**Selección de atributos:**

De acuerdo al análisis se determina:

- No utilizar el atributo "Relación_en_la_familia".
- No utilizar el atributo "Educación". 
- No utilizar el atributo "Peso_final". 

Se realizarán ejecuciones sin tomar en cuenta también los atributos Raza y País_Natal, dada su baja correlación con la clase de salida. 

**Representación de atributos:**

Para la representación de los atributos categóricos con más de dos categorías utilizaremos one-hot-encoding. Se elije esta opción sobre la de asignar un valor numérico a cada categoría ya que el algoritmo KNN podría verse perjudicado al asumir orden entre las diferentes etiquetas e incidir en el cálculo de las distancias. 
Para la representación del atributo Sexo, se determina colocar 2 etiquetas 0 y 1, ya que al tener solo dos opciones, no incidirá en los algoritmos. 

La clase de salida tiene solamente dos valores, por lo que lo transformaremos en valores numéricos binarios dado que Scikit-learn solamente acepta atributos numéricos. O para <=50K, y 1 para >50K. 

**Tratamiento de elementos faltantes:**

Para el caso de los atributos faltantes del atributo "Relación laboral", se determina colocar el atributo más común de la clase, que es "Private" ya que casi el 50% de los registros tienen este valor. Para el caso del atributo "País Natal" se determina realizar el mismo método colocando el valor "United States" ya que casi el 60% de los registros tienen ese valor. para el caso del atributo "ocupación", no hay un valor que sea ampliamente mayoritario, por lo que se decide quitar del dataset todos los registros que tengan valores desconocidos para este atributo (3,7% del dataset). 
Al tomar esta última decisión, y realizar un análisis del dataset, vamos a encontrar que todos los registros con datos faltantes referidos a la Relación laboral se eliminarán, por lo que no hará falta colocar el valor "Private" a los atributos desconocidos. 

**Estandarización de atributos:**

Se elije utilizar el método de min-max scaling para la estandarización de atributos numéricos que no estén en el rango de 0-1. De esta manera todos los atributos tendrán el mismo orden de magnitud, y ningún atributo tendrá más peso que otro al calcular las distancias.

Se muestra a continuación el perfil del dataset luego de procesado:



In [2]:
dataset_resultado = preprocesamiento.preprocesamiento()
pandas_profiling.ProfileReport(dataset_resultado).to_widgets()


    
## 3 Parte B

## 3.1 División del Conjunto de Datos

Se generarán los conjuntos de entrenamiento y test realizando una división del dataset de 80%-20%.
Para el ajuste de los parámetros del modelo se utilizará validación cruzada, por lo que no es necesario separar del  conjunto de entrenamiento un conjunto de validación.

Dado que las clases de salida se encuentran desbalanceadas (75%, 25%), se determina utilizar estratificación para asegurarnos que tendremos la misma proporción de ambas clases en todos los conjuntos. También se determina no utilizar una semilla (variable random_state), para que los datasets sean diferentes para cada ejecución.
Para realizar los diferentes conjuntos, ejecutamos el siguiente código. 


In [3]:
#creamos conjuntos entrenamiento y test (entrenamiento 80% - test 20%)

train, test = model_selection.train_test_split(dataset_resultado, stratify=dataset_resultado['Salida'], test_size=0.2)

Para realizar la validación cruzada de los algoritmos, se utiliza el método model_selection.KFold. Para cada división del conjunto de entrenamiento en subconjuntos Train-Test, se ejecuta el algoritmo para predecir, y se calculan las métricas de accurancy, precisión, recall y F1. Luego se hace un promedio de las métricas obtenidas en cada una de las iteraciones, siendo éstas las métricas de performance del modelo. 
Utilizaremos este método para encontrar los mejores parámetros para nuestros algoritmos, y luego de tenerlos, evaluaremos contra el conjunto de test la performance final. 

## 3.2 Evaluación

Se utilizan las métricas: accurancy, precisión, recall y F1. 

Precisión y Recuperación nos ayudan a determinar el comportamiento para cada clase de salida.
La precisión mide qué tan bueno es el clasificador cuando dice que un ejemplo es de una determinada clase, mientras que la recuperación mide qué proporción encuentra de los elementos de una clase existentes. La medida-F es la media armónica entre precisión y recall, e intenta combinar ambas en un sólo número. tomaremos F con $\beta$ = 1. 

Calcularemos precisión, recall y F1 para ambas clases de salida, utilizando los métodos de Scikit-learn.


    
## 4 Parte C

## 4.1 Algoritmo KNN

Se implementa el algoritmo sin modificaciones al original, con la posibilidad de elegir la cantidad de vecinos. 

Para el cálculo de la distancia entre dos ejemplos se utiliza la norma del vector diferencia entre los ejemplos.
La norma de un vector $\Vert v\Vert$ es una medida del "tamaño" de un vector. La norma euclídea ($\ell_2$), equivalente a $\sqrt{x^Tx}$ (puede verse que $||x||^2_2 = x^Tx$):

$$ \Vert x \Vert_2 = \sqrt{\sum_{i=1}^n x_i^2}$$

si x es el vector diferencia entre los dos ejemplos, tenemos la distancia entre ellos. 

Con Numpy puede calcularse facilmente con la función (por defecto es la norma ($\ell_2$) ) numpy.linalg.norm

Para la selección de los vecinos más cercanos, una vez que se tiene el vector de distancias, se ordena (con el método numpy.argsort) y se eligen los k primeros. 

En la sección de experimentación se da la posibilidad de ejecución del algoritmo.




## 4.2 Ejecución de KNN

Aquí se da la posibilidad de ejecutar la implementación del algoritmo KNN. Se permite elegir la cantidad de vecinos a tener en cuenta y el tamaño del dataset para poder correrlo en menor tiempo. 

In [7]:
# aquí se puede seleccionar los vecinos a tener en cuenta y la cantidad de ejemplos de entrenamiento y testeo
vecinos = 10
CantidadEjemplosEntrenamiento = train.shape[0]
CantidadinstanciasEvaluar = test.shape[0]

KNN(train, test, vecinos, CantidadEjemplosEntrenamiento, CantidadinstanciasEvaluar)

              precision    recall  f1-score   support

           0       0.86      0.92      0.89      4639
           1       0.68      0.53      0.60      1505

    accuracy                           0.82      6144
   macro avg       0.77      0.72      0.74      6144
weighted avg       0.81      0.82      0.82      6144



## 4.3 Resultados de ejecución 

Se presentan los resultados de la ejecución con diferentes valores de K, cantidad de ejemplos y cantidad de splits para la validación cruzada. 

<table>
  <tr>
    <th># vecinos</th>
    <th># Ejemplos</th> 
    <th># Splits</th> 
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
  <tr>
    <td>3</td>
    <td>|train|</td>
    <td>5</td>
    <td>0,81</td>
    <td>0,64</td>
    <td>0,57</td>
    <td>0,60</td>
    <td>0,86</td>
    <td>0,89</td>
    <td>0,88</td>
  </tr>
  <tr>
    <td>5</td>
    <td>|train|</td>
    <td>5</td>
    <td>0,82</td>
    <td>0,66</td>
    <td>0,57</td>
    <td>0,62</td>
    <td>0,87</td>
    <td>0,90</td>
    <td>0,88</td>
  </tr> 
    <tr>
    <td>7</td>
    <td>|train|</td>
    <td>5</td>
    <td>0,82</td>
    <td>0,66</td>
    <td>0,57</td>
    <td>0,62</td>
    <td>0,87</td>
    <td>0,90</td>
    <td>0,88</td>
  </tr> 
  <tr style="font-weight:bold">
    <td>10</td>
    <td>|train|</td>
    <td>5</td>
    <td>0,83</td>
    <td>0,71</td>
    <td>0,54</td>
    <td>0,61</td>
    <td>0,86</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
  <tr>
    <td>10</td>
    <td>|train|</td>
    <td>10</td>
    <td>0,83</td>
    <td>0,71</td>
    <td>0,52</td>
    <td>0,60</td>
    <td>0,85</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
  <tr>
    <td>10</td>
    <td>|train|/2</td>
    <td>5</td>
    <td>0,83</td>
    <td>0,72</td>
    <td>0,54</td>
    <td>0,62</td>
    <td>0,86</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
    <tr>
    <td>10</td>
    <td>|train|/4</td>
    <td>5</td>
    <td>0,83</td>
    <td>0,70</td>
    <td>0,52</td>
    <td>0,60</td>
    <td>0,85</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
  <tr>
    <td>10</td>
    <td>1000</td>
    <td>5</td>
    <td>0,80</td>
    <td>0,66</td>
    <td>0,44</td>
    <td>0,52</td>
    <td>0,83</td>
    <td>0,92</td>
    <td>0,87</td>
  </tr>
  <tr>
    <td>20</td>
    <td>|train|</td>
    <td>5</td>
    <td>0,83</td>
    <td>0,70</td>
    <td>0,54</td>
    <td>0,61</td>
    <td>0,86</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
    <tr>
    <td>21</td>
    <td>|train|</td>
    <td>5</td>
    <td>0,83</td>
    <td>0,70</td>
    <td>0,56</td>
    <td>0,62</td>
    <td>0,86</td>
    <td>0,92</td>
    <td>0,89</td>
  </tr>
    <caption>Tabla 1 - Comparación algoritmos KNN</caption>
    <caption>Nota: |train|=24574</caption>
    
</table>



Se observa que la performance del algoritmo es similar para todos los casos, cambiando la cantidad de vecinos. Se realizó la prueba de realizar más splits en la validación cruzada, para disminuir la varianza de los resultados, pero los resultados también fueron similares. 

Observando las métricas, se determina que 10 sería un buen valor para k. Para ese caso, se realiza también la prueba de ejecutar el algoritmo con menos ejemplos de entrenamiento, obteniendo sorpresivamente resultados muy similares para datasets 75% más pequeños. Se realizó la prueba con 1000 ejemplos y la performance es apenas más baja. Esta prueba es interesante ya que con datasets mucho más pequeños, (que implica menor costo computacional), se obtienen resultados muy similares.

Con respecto al dataset, recordemos que teníamos 2 columnas, Capital Ganado y Capital perdido, que tenían cerca de un 95% de valores en cero. Decidimos hacer una prueba descartando dichas columnas para evaluar si mejora la performance. También se realizó la prueba no teniendo en cuenta las columnas de Raza y País_natal, que según el reporte no estaban correlacionadas a la clase salida. 

Los resultados fueron similares a nivel de performance para ambas pruebas pero no así a nivel de costo computacional.  Teniendo en cuenta las columnas de Raza y País natal, y realizando one-hot-encoding de ellas, la validación cruzada demoró 1109 segundos, mientras que sin esas columnas demoró 450 segundos. Recordemos que al realizar el hot-encoding de estas dos columnas, agregamos casi 46 columnas adicionales. Al no tener en cuenta estos datos, nos estamos quedando aproximadamente con un 43% de las columnas. Podemos ver que el tiempo que lleva la tarea también se reduce en un número similar (37,5%). 

<table>
  <tr>
    <th>cambios</th>
    <th># vecinos</th>
    <th># Ejemplos</th> 
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
  <tr>
    <td>Sin columnas Capital Ganado y Perdido</td>
    <td>10</td>
    <td>|train|</td>
    <td>0,83</td>
    <td>0,71</td>
    <td>0,56</td>
    <td>0,63</td>
    <td>0,86</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
 <tr style="font-weight:bold">
    <td>Sin columnas Raza y País Natal</td>
    <td>10</td>
    <td>|train|</td>
    <td>0,83</td>
    <td>0,72</td>
    <td>0,54</td>
    <td>0,61</td>
    <td>0,85</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
  <tr>
    <td>Sin columnas Raza, País Natal, Capital Ganado y Perdido</td>
    <td>10</td>
    <td>|train|</td>
    <td>0,82</td>
    <td>0,68</td>
    <td>0,54</td>
    <td>0,60</td>
    <td>0,89</td>
    <td>0,83</td>
    <td>0,86</td>
  </tr>  
     <caption>Tabla 2 - KNN con 10 vecinos con validación cruzada y sin columnas de capital ganado y perdido</caption>
</table>

En este caso observamos que quitar las columnas que representan los atributos de Raza y País Natal hace una diferencia sustancial en los tiempos de ejecución del algoritmo, que ya de por si demora en el orden de los minutos. Por otro lado quitar también las columnas de Capital ganado y perdido hace que la performance disminuya (aunque mínimamente, igual disminuye). 

Luego de haber detectado los hiperparámetros que mejor funcionan para el algoritmo, e intentando hacer una balance entre performance y eficiencia computacional,  se decide ejecutar contra el conjunto de Test, sin considerar las columnas de Raza y País_Natal, pero si las de capital ganado y perdido. Se obtienen los siguientes resultados:

<table>
  <tr>
    <th># vecinos</th>
    <th># Ejemplos</th> 
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
    <th>tiempo ejecución (seg)</th>
  </tr>
  <tr>
    <td>10</td>
    <td>|train|</td>
    <td>0,85</td>
    <td>0,76</td>
    <td>0,57</td>
    <td>0,65</td>
    <td>0,87</td>
    <td>0,94</td>
    <td>0,90</td>
    <td>149</td>
  </tr>
     <caption>Tabla 3 - KNN con 10 vecinos contra conjunto de test</caption>
</table>

## 5. Parte D

    
## 5.1 Algoritmo Naive Bayes






Se implementa el algoritmo visto en el teórico, realizando primeramente un cálculo de las probabilidades de todas las clases de los atributos, dadas las diferentes salidas. Luego se evalúan las instancias utilizando dicha información. 

**Decisiones para el cálculo de las probabilidades:**

Se decidió utilizar m-estimador para resolver 2 problemas:
* cuando las clases de los atributos están muy desbalanceadas, y hay pocos ejemplos de una clase, la estimación de las probabilidades puede ser mala. Por ejemplo, para la columna "Cambodia" donde hay 18 valores positivos y 30700 negativos, calcular las probabilidades sin m-estimador, nos dará valores pobres. Esto mismo nos sucede con varias de las columnas de nuestro dataset.
* cuando en el conjunto de entrenamiento que se utiliza para calcular las probabilidades no está representada alguna clase de un atributo, la probabilidad sin m-estimador es cero. Cuando estamos prediciendo la probabilidad para una instancia, y nos topamos con una de estas probabilidades con valor cero, hace que todo el término sea cero, por lo que puede inducir una generalización errónea. 

para el m-estimador asumimos que todos los atributos son equiprobables en cada clase, por lo que para las clases binarias p= 0.5 y para las continuas p=0.25 (explicaremos a continuación el manejo de las clases continuas y el porqué de este p).

**Manejo de atributos con valores continuos:**

Para el cálculo de las probabilidades en el caso de los atributos con valores continuos debe realizarse algún tipo de categorización, ya que no tiene sentido calcular la probabilidad de cada uno de los posibles valores. En nuestro caso, luego del preprocesamiento los valores continuos se encuentran entre 0 y 1. Se determina dividir los valores en 4 categorías:
* (x<0.25)
* (x>=0.25) & (x<0.50)
* (x>=0.50) & (x<0.75)
* (x>0.75)

Asumiendo que son equiprobables, p de m-estimador es 1/4= 0.25 

**Estructura de datos para guardar probabilidades:**

Se utilizan diccionarios anidados de dos niveles, donde las claves del diccionario de primer nivel son los nombres de las columnas. Luego, para una columna cuyos valores son binarios se tendrán diccionarios con 4 ítems, con claves: P0|salida0, P0|salida1, P1|salida0 y P1|salida1. El ítem cuya clave es P0|salida0 es la probabilidad que dado el valor 0 en la salida, el atributo tenga valor 0. Análogamente se explican las demás claves.

Para el caso de las columnas con valores continuos, se tendrán diccionarios con 8 items con claves: P1cuarto|salida0, P1cuarto|salida1, P2cuarto|salida0, P2cuarto|salida1, P3cuarto|salida0, P3cuarto|salida1, P4cuarto|salida0 y P4cuarto|salida1. El item cuya clave es P1cuarto|salida0 es la probabilidad que dado el valor 0 en la salida, el atributo tenga valor menor a 0.25 (1er cuarto). Análogamente se explican las demás claves.

**Estimación de una probabilidad:**

Teniendo la estructura de probabilidades y la probabilidad de las clases de salida, calculadas previamente, se toma el ejemplo a estimar. Para cada ejemplo se calculan los 2 términos de productos de probabilidades, uno para cuando la salida es 0 y otro para cuando es 1. se itera en las columnas, evaluando el valor del ejemplo y determinando cual probabilidad escoger del diccionario de la columna. Luego se devuelve el valor para el cual el término calculado es mayor. Aquí solamente se predice un valor con el que tiene la probabilidad mayor, pero podría también interesar saber con exactitud cual es la probabilidad que el valor estimado sea el correcto, información que tenemos y podríamos exponer. 



## 5.2 Ejecución de Naive Bayes

Aquí se da la posibilidad de ejecutar la implementación del algoritmo Naive Bayes. Se permite elegir el valor m para el m-estimador y el tamaño de los datasets. 

In [6]:
# aquí se puede seleccionar m, la cantidad de ejemplos de entrenamiento y testeo
m = 0.1
CantidadEjemplosEntrenamiento = train.shape[0]
CantidadinstanciasEvaluar = test.shape[0]

NB(train, test, CantidadEjemplosEntrenamiento, CantidadinstanciasEvaluar, m)

              precision    recall  f1-score   support

           0       0.89      0.83      0.86      4607
           1       0.58      0.70      0.64      1537

    accuracy                           0.80      6144
   macro avg       0.74      0.77      0.75      6144
weighted avg       0.82      0.80      0.81      6144



## 5.3 Resultados de ejecución 

Se presentan los resultados de la ejecución con diferentes valores de m, cantidad de ejemplos y cantidad de splits para la validación cruzada. 

<table>
  <tr>
    <th>m</th>
    <th># Ejemplos</th> 
    <th># Splits</th> 
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
 <tr style="font-weight:bold">
    <td>0.1</td>
    <td>|train|</td>
    <td>5</td>
    <td>0,80</td>
    <td>0,57</td>
    <td>0,71</td>
    <td>0,64</td>
    <td>0,90</td>
    <td>0,83</td>
    <td>0,86</td>
  </tr> 
    <tr>
    <td>0.1</td>
    <td>|train|/2</td>
    <td>5</td>
    <td>0,79</td>
    <td>0,57</td>
    <td>0,73</td>
    <td>0,64</td>
    <td>0,90</td>
    <td>0,82</td>
    <td>0,85</td>
  </tr> 
        <tr>
    <td>0.1</td>
    <td>|train|/4</td>
    <td>5</td>
    <td>0,78</td>
    <td>0,55</td>
    <td>0,72</td>
    <td>0,63</td>
    <td>0,89</td>
    <td>0,80</td>
    <td>0,84</td>
  </tr> 
  <tr>
    <td>0.3</td>
    <td>|train|</td>
    <td>5</td>
    <td>0,80</td>
    <td>0,60</td>
    <td>0,64</td>
    <td>0,62</td>
    <td>0,88</td>
    <td>0,86</td>
    <td>0,87</td>
  </tr> 
  <tr>
    <td>0.5</td>
    <td>|train|</td>
    <td>5</td>
    <td>0,81</td>
    <td>0,64</td>
    <td>0,54</td>
    <td>0,59</td>
    <td>0,86</td>
    <td>0,90</td>
    <td>0,88</td>
  </tr>
  <tr>
    <td>0.7</td>
    <td>|train|</td>
    <td>5</td>
    <td>0,81</td>
    <td>0,71</td>
    <td>0,40</td>
    <td>0,51</td>
    <td>0,83</td>
    <td>0,95</td>
    <td>0,88</td>
  </tr>
    <tr>
    <td>1</td>
    <td>|train|</td>
    <td>5</td>
    <td>0,80</td>
    <td>0,78</td>
    <td>0,29</td>
    <td>0,41</td>
    <td>0,80</td>
    <td>0,97</td>
    <td>0,88</td>
  </tr> 
  <tr>
    <td>2 y 3</td>
    <td>|train|</td>
    <td>5</td>
    <td>0,75</td>
    <td>0</td>
    <td>0</td>
    <td>N/A</td>
    <td>0,75</td>
    <td>1</td>
    <td>0,85</td>
  </tr>
    <caption>Tabla 4 - Comparación algoritmos NB</caption>
    <caption>Nota: |train|=24574</caption>
    
</table>

Si bien los resultados son similares, vemos que a medida que aumentamos el factor m, los resultados para la clase más pequeña van siendo peores. Vemos también que los mejores resultados de accurancy se obtienen con m entre 0,5 y 0,7, luego comienza a decrecer. 
Dado que nos interesa tener los mejores resultados para la clase positiva, elegimos como mejor caso el de m=0,1. La validación cruzada para este caso demora 33,7 segundos. 

Ejecutaremos a continuación la validación cruzada realizando eliminando atributos, de la misma forma que se realizó para KNN. Los resultados son los siguientes:


<table>
  <tr>
    <th>cambios</th>
    <th># m</th>
    <th># Ejemplos</th> 
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
    <th>tiempo ejecución</th>
  </tr>
  <tr>
    <td>Sin columnas Capital Ganado y Perdido</td>
    <td>0.1</td>
    <td>|train|</td>
    <td>0,79</td>
    <td>0,56</td>
    <td>0,73</td>
    <td>0,64</td>
    <td>0,90</td>
    <td>0,81</td>
    <td>0,85</td>
    <td>33,7</td>
  </tr>
  <tr>
    <td>Sin columnas Raza y País Natal</td>
    <td>0.1</td>
    <td>|train|</td>
    <td>0,80</td>
    <td>0,58</td>
    <td>0,69</td>
    <td>0,63</td>
    <td>0,88</td>
    <td>0,83</td>
    <td>0,86</td>
    <td>20,82</td>
  </tr>
  <tr>
    <td>Sin columnas Raza, País Natal, Capital Ganado y Perdido</td>
    <td>0.1</td>
    <td>|train|</td>
    <td>0,80</td>
    <td>0,58</td>
    <td>0,69</td>
    <td>0,63</td>
    <td>0,89</td>
    <td>0,83</td>
    <td>0,86</td>
    <td>20,2</td>
  </tr>
 <caption>Tabla 5 - NB con m=0.1 contra conjunto de test</caption>
</table>


Vemos que los tiempos de ejecución disminuyen un 50% cuando quitamos los datos de Raza y País_natal. La performance en la clase positiva disminuye, pero considerablemente poco. 

Aquí tenemos para plantearnos: ¿Se prioriza mejor performance a cualquier costo? ¿Cuán mejor deben ser los resultados para yo elegir una solución con mayor costo computacional frente a otra? Si bien bajan los tiempos de ejecución, estamos hablando del orden de los segundos... ¿Afecta que mi algoritmo demore 10 segundos más? Las respuestas pueden ser varias, y claramente depende del contexto. Para este caso mi decisión es elegir la solución con mayor performance en las clases positivas, por lo que para evaluar el modelo no quitaré las columnas Raza, País Natal, Capital Ganado y Perdido.

Luego de haber detectado los hiperparámetros que mejor funcionan para el algoritmo, se ejecuta contra el conjunto de Test, obteniéndose los siguientes resultados:

<table>
  <tr>
    <th>m</th>
    <th># Ejemplos</th> 
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
  <tr>
    <td>0.1</td>
    <td>|train|</td>
    <td>0,79</td>
    <td>0,55</td>
    <td>0,72</td>
    <td>0,62</td>
    <td>0,90</td>
    <td>0,81</td>
    <td>0,85</td>
  </tr>
     <caption>Tabla 6 - NB con m=0.1 contra conjunto de test</caption>
</table>




## 6. Parte E

## 6.1 Experimentación con Scikit Learn para KNN

Se presentarán a continuación los resultados de las implementación de Scikit-learn para KNN variando algunos de los hiperparámetros. Cabe destacar que se eligió jugar con el algoritmo de determinación de los vecinos y con la cantidad de vecinos. No se probó con otras métricas de distancia, aunque el método lo permite. 


<table>
   <tr>
    <th># vecinos</th>
    <th># Ejemplos</th> 
    <th># Splits</th> 
    <th># Algoritmo</th>
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
  <tr>
    <td>10</td>
    <td>|train|</td>
    <td>5</td>
    <td>auto</td>
    <td>0,83</td>
    <td>0,72</td>
    <td>0,52</td>
    <td>0,60</td>
    <td>0,85</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
  <tr>
    <td>10</td>
    <td>|train|</td>
    <td>5</td>
    <td>ball_tree</td>
    <td>0,83</td>
    <td>0,71</td>
    <td>0,53</td>
    <td>0,61</td>
    <td>0,86</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
  <tr>
    <td>10</td>
    <td>|train|</td>
    <td>5</td>
    <td>kd_tree</td>
    <td>0,83</td>
    <td>0,71</td>
    <td>0,53</td>
    <td>0,61</td>
    <td>0,86</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
  <tr>
    <td>10</td>
    <td>|train|</td>
    <td>5</td>
    <td>brute</td>
    <td>0,83</td>
    <td>0,71</td>
    <td>0,52</td>
    <td>0,60</td>
    <td>0,85</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
  <tr>
    <td>20</td>
    <td>|train|</td>
    <td>5</td>
    <td>auto</td>
    <td>0,83</td>
    <td>0,71</td>
    <td>0,54</td>
    <td>0,62</td>
    <td>0,86</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
  <tr>
    <td>5</td>
    <td>|train|</td>
    <td>5</td>
    <td>auto</td>
    <td>0,82</td>
    <td>0,65</td>
    <td>0,57</td>
    <td>0,61</td>
    <td>0,86</td>
    <td>0,90</td>
    <td>0,88</td>
  </tr>
    <caption>Tabla 7 - experimentación con KNN de Scikit-learn</caption>
</table>

Observando la tabla vemos que para todos los casos el algoritmo tiene peor performance para detectar los casos positivos. Esto se da por la naturaleza del dataset, donde una gran proporción de los ejemplos son de clase positiva en la salida. Observamos que el desempeño es muy similar para todos los casos. 


## 6.2 Comparación KNN implementado contra KNN Scikit Learn

Elegimos la mejor combinación de parámetros para nuestra implementación, y elegimos uno de los casos de Scikit-learn, ya que son todos tienen performances muy similares.

<table>
   <tr>
    <th>Implementación</th> 
    <th># vecinos</th>
    <th># Ejemplos</th>   
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
  <tr>
    <td>SciKit-learn</td>
    <td>10</td>
    <td>|train|</td>
    <td>0,83</td>
    <td>0,72</td>
    <td>0,52</td>
    <td>0,60</td>
    <td>0,85</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
  <tr>
    <td>Parte C</td>
    <td>10</td>
    <td>|train|</td>
    <td>0,83</td>
    <td>0,72</td>
    <td>0,54</td>
    <td>0,61</td>
    <td>0,85</td>
    <td>0,93</td>
    <td>0,89</td>
  </tr>
        <caption>Tabla 8 - comparación algoritmo implementado contra versión de Scikit-learn </caption>
</table>

Se observa que la performance de ambas implementaciones es prácticamente la misma. 



## 6.3 Experimentación con Scikit Learn para Naive Bayes

Se presentarán a continuación los resultados de las implementación de Scikit-learn para Naive Bayes variando algunos de los hiperparámetros. 
Existen diversos clasificadores de Naive Bayes implementados en la librería Scikit-learn, que asumen diversas premisas sobre los datos. Explicaremos muy brevemente cada uno y porqué decidimos experimentar o no con él. 

GaussianNB: asume que los datos de cada atributo tienen una distribución Gaussiana. En nuestro caso no lo tiene, por lo que dicho clasificador no nos sirve, pero decidimos probarlo igualmente solo por curiosidad. los resultados no son buenos.

MultinomialNB: asume una distribución multinomial de los datos. Se utiliza mucho en la clasificación de textos. Para probar los resultados de este clasificador, se realiza un procesamiento adicional de los datos, modificando las columnas con valores continuos y colocándoles categóricos (4 valores posibles). se prueba con valores entre 0 y 1, y también con valores 1,2,3 y 4 ya que la documentación indica que funciona mejor con valores enteros aunque también puede funcionar con fracciones. se realizaron ambas pruebas y los resultados son los mismos. 

ComplementNB: es una adaptación de MultinomialNB, que funciona mejor cuando las clases están desbalanceadas. Se realiza la prueba con el procesamiento adicional. 

BernoulliNB: asume una distribución de bernoulli sobre los datos de los atributos. Esto requiere que los atributos sean valores booleanos. Para los valores que no son, se puede utilizar el parámetro binarize, que los transforma a 0 y 1. como nuestro dataset tiene casi todos valores binarios, probamos este clasificador con el parámetro binarize con diferentes valores. 


<table>
   <tr>
    <th>$\alpha$</th>
    <th># Ejemplos</th> 
    <th># Splits</th> 
    <th># Algoritmo</th>
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
  <tr>
    <td>1</td>
    <td>|train|</td>
    <td>5</td>
    <td>Gaussian</td>
    <td>0,33</td>
    <td>0,27</td>
    <td>0,97</td>
    <td>0,42</td>
    <td>0,93</td>
    <td>0,12</td>
    <td>0,22</td>
  </tr>
  <tr>
    <td>1</td>
    <td>|train|</td>
    <td>5</td>
    <td>Complement</td>
    <td>0,75</td>
    <td>0,49</td>
    <td>0,82</td>
    <td>0,61</td>
    <td>0,92</td>
    <td>0,72</td>
    <td>0,80</td>
  </tr>
  <tr>
    <td>0.5</td>
    <td>|train|</td>
    <td>5</td>
    <td>Complement</td>
    <td>0,75</td>
    <td>0,49</td>
    <td>0,81</td>
    <td>0,61</td>
    <td>0,92</td>
    <td>0,72</td>
    <td>0,81</td>
  </tr>
  <tr>
    <td>1</td>
    <td>|train|</td>
    <td>5</td>
    <td>Multinomial</td>
    <td>0,80</td>
    <td>0,62</td>
    <td>0,52</td>
    <td>0,56</td>
    <td>0,85</td>
    <td>0,89</td>
    <td>0,87</td>
  </tr>
  <tr>
    <td>0.5</td>
    <td>|train|</td>
    <td>5</td>
    <td>Multinomial</td>
    <td>0,80</td>
    <td>0,62</td>
    <td>0,52</td>
    <td>0,57</td>
    <td>0,85</td>
    <td>0,90</td>
    <td>0,87</td>
  </tr>
  <tr style="font-weight:bold">
    <td>1</td>
    <td>|train|</td>
    <td>5</td>
    <td>Bernoulli 0.5</td>
    <td>0,77</td>
    <td>0,53</td>
    <td>0,73</td>
    <td>0,61</td>
    <td>0,89</td>
    <td>0,78</td>
    <td>0,84</td>
  </tr>
  <tr style="font-weight:bold">
    <td>0.5</td>
    <td>|train|</td>
    <td>5</td>
    <td>Bernoulli 0.5</td>
    <td>0,77</td>
    <td>0,53</td>
    <td>0,73</td>
    <td>0,61</td>
    <td>0,89</td>
    <td>0,79</td>
    <td>0,84</td>
  </tr>
    <tr>
    <td>0.5</td>
    <td>|train|</td>
    <td>5</td>
    <td>Bernoulli 1</td>
    <td>0,75</td>
    <td>0</td>
    <td>0</td>
    <td>N/A</td>
    <td>0,75</td>
    <td>1</td>
    <td>0,86</td>
  </tr>
      <tr>
    <td>0.5</td>
    <td>|train|</td>
    <td>5</td>
    <td>Bernoulli 0.9</td>
    <td>0,77</td>
    <td>0,53</td>
    <td>0,73</td>
    <td>0,61</td>
    <td>0,89</td>
    <td>0,79</td>
    <td>0,84</td>
  </tr>
    <caption>Tabla 9 - experimentación con NB de Scikit-learn</caption>
</table>

Observando la tabla vemos que descartando GaussianNB, todos los algoritmos tienen un comportamiento similar. Observando la tabla, se ve que el Clasificador de Bernoulli, binarizando con 0.5 tiene buenos resultados. En nuestro caso le estamos dando mayor importancia a buena performance en la clase positiva, más que accurancy o performance en la clase negativa. Para el caso del clasificador de Bernoulli, no se perciben cambios en la performance cambiando $\alpha$. 


## 6.4 Comparación Naive Bayes implementado contra Naive Bayes Scikit Learn

Elegimos la mejor combinación de parámetros para nuestra implementación, y elegimos el mejor de los casos de Scikit-learn:

<table>
   <tr>
    <th>Implementación</th> 
    <th>m </th>
    <th># Ejemplos</th>   
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
  <tr>
    <td>Scikit Bernoulli 0.5</td>
    <td>0.5 </td>
    <td>|train|</td>
    <td>0,77</td>
    <td>0,53</td>
    <td>0,73</td>
    <td>0,61</td>
    <td>0,89</td>
    <td>0,78</td>
    <td>0,84</td>
  </tr>
  <tr>
    <td>Parte D</td>
    <td>0.1</td>
    <td>|train|</td>
    <td>0,80</td>
    <td>0,57</td>
    <td>0,71</td>
    <td>0,64</td>
    <td>0,90</td>
    <td>0,83</td>
    <td>0,86</td>
  </tr>
        <caption>Tabla 10 - comparación algoritmo implementado contra versión de Scikit-learn</caption>
</table>

Se observa que la performance del algoritmo implementado es un tanto mejor que Bernoulli. Suponemos que el algoritmo implementado tiene mejores resultados dado que los atributos continuos se partieron en 4 categorías, y para el clasificador de Bernoulli se dividen solamente en 2. 


## 7. Parte F

## 7.1 Resultados algoritmos contra conjunto de test

A continuación se presentan los resultados de la ejecución contra el conjunto de test de los algoritmos implementados y de los algoritmos de Scikit con los mejores hiperparámetros encontrados. 

<table>
  <tr>
    <th>Algoritmo</th>
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
    <th>Tiempo ejecución (seg.)</th>
  </tr>
  <tr>
    <td>KNN implementado</td>
    <td>0,85</td>
    <td>0,76</td>
    <td>0,57</td>
    <td>0,65</td>
    <td>0,87</td>
    <td>0,94</td>
    <td>0,90</td>
    <td>149</td>
  </tr>
  <tr>
    <td>KNN SciKit</td>
    <td>0,86</td>
    <td>0,78</td>
    <td>0,59</td>
    <td>0,67</td>
    <td>0,87</td>
    <td>0,94</td>
    <td>0,91</td>
    <td>10</td>
  </tr>
  <tr>
    <td>NB SciKit - Bernoulli</td>
    <td>0,77</td>
    <td>0,53</td>
    <td>0,74</td>
    <td>0,61</td>
    <td>0,90</td>
    <td>0,78</td>
    <td>0,84</td>
    <td>0,1</td>
  </tr>
  <tr>
    <td>NB implementado</td>
    <td>0,79</td>
    <td>0,55</td>
    <td>0,72</td>
    <td>0,62</td>
    <td>0,90</td>
    <td>0,81</td>
    <td>0,85</td>
    <td>8,6</td>
  </tr>
     <caption>Tabla 11 - Resultados implementaciones y Scikit-learn contra el conjunto de test</caption>
</table>

## 4. Conclusión

En general, los algoritmos tienen resultados globales de performance similares, si miramos Accurancy, y la medida F para las clases de salida. Para la clase con menos ejemplos, la clase positiva, se tiene una menor performance que para la clase negativa. Se observa también que KNN tiene mejor precisión en la clase positiva, y mejor recall en clase negativa que Naive Bayes, mientras que este último tiene los valores inversos. 

¿Qué quiere decir esto? 

KNN cuando predice que un ejemplo es positivo, se equivoca menos que NB. Sin embargo NB tiene mejor capacidad para encontrar los ejemplos positivos, aunque a algún negativo lo clasifique como positivo. 

¿Cuál algoritmo funciona mejor? 

Dependerá del contexto en el que estemos. Si yo quiero predecir cuando una persona gana más de 50.000 USD anuales para enviarle propagandas de artículos lujosos, quizá no interese que me equivoque y le mande a alguno que no perciba esos ingresos. En ese caso elegiré Naive Bayes. Ahora, si lo que quiero es predecir para cobrarles un impuesto adicional a quienes perciban más de 50.000 USD, intentaré equivocarme lo menos posible y elegiré KNN. 

Tiempos de ejecución:

Los tiempos para los algoritmos de Scikit-learn son en todos los casos mejores. Comparando entre algoritmos, vemos que Naive Bayes es extremadamente rápido, dando resultados de performance muy similares a KNN. 
En nuestra experimentación también aprendimos que la elección de los atributos relevantes es fundamental para obtener buenos resultados con menor costo computacional, pudiendo obtenerse reducciones muy significativas de tiempo sin sacrificar performance.
